In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('product_database.csv')

In [2]:
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D

# Load the CSV file
file_path = 'product_database.csv'
product_data = pd.read_csv(file_path)

# Combine 'Product_Name' and 'About_Product' columns for text input
product_data['combined_text'] = product_data['Product_Name'] + " " + product_data['About_Product']

# Function to clean text
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Apply the clean_text function to the combined text
product_data['cleaned_text'] = product_data['combined_text'].apply(clean_text)


In [3]:



# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(product_data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(product_data['cleaned_text'])


print(product_data['cleaned_text'].head())
print(sequences[:5])
# Pad the sequences


0    wayona nylon braided usb to lightning fast cha...
1    ambrane unbreakable w  a fast charging m braid...
2    sounce fast phone charging cable  data sync us...
3    boat deuce usb   in  typec  micro usb stress r...
4    portronics konnect l m fast charging a  pin us...
Name: cleaned_text, dtype: object
[[851, 187, 137, 11, 3, 278, 23, 18, 1, 41, 74, 13, 30, 5, 78, 28, 64, 142, 56, 108, 555, 354, 7, 295, 53, 220, 30, 4, 78, 2522, 78, 1134, 1134, 896, 1134, 1134, 2742, 2743, 142, 64, 108, 677, 2744, 97, 1, 68, 252, 2341, 1643, 74, 15, 17, 42, 1, 74, 678, 45, 6, 530, 46, 30, 4, 48, 18, 1102, 1201, 18, 1382, 16, 26, 2037, 100, 187, 137, 34, 4, 165, 512, 556, 1, 1898, 187, 814, 2038, 1279, 468, 2, 128, 1899, 15, 480, 370, 1, 1325, 6, 751, 3, 124, 2039, 828, 385, 15, 9, 156, 3, 611, 406, 8, 83, 19, 1383, 1384, 1900, 304, 1806, 951, 1807, 589, 1503, 2523, 313, 24, 1, 481, 225, 1202, 212, 2, 2040, 2041, 7, 8, 211, 246, 12, 1135, 48, 1901, 16, 1003, 106, 626, 1236, 3, 269, 494], [852, 737,

In [4]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Prepare labels (we can use Product_ID as dummy labels for now)
labels = product_data['Product_ID']

# Define the model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(len(product_data['Product_ID'].unique()), activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(padded_sequences, labels, epochs=10, batch_size=32, validation_split=0.2)
# Save the model
model.save('product_similarity_model.h5')


Epoch 1/5
37/37 [==============================] - 1s 14ms/step - loss: 7.2987 - accuracy: 8.5324e-04 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/5
37/37 [==============================] - 0s 7ms/step - loss: 7.2814 - accuracy: 8.5324e-04 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/5
37/37 [==============================] - 0s 5ms/step - loss: 7.2666 - accuracy: 0.0017 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/5
37/37 [==============================] - 0s 5ms/step - loss: 7.2372 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/5
37/37 [==============================] - 0s 5ms/step - loss: 7.1914 - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
